In [8]:
# Declarations
import glob
import json
import os
import numpy as np
import pandas as pd
import re

from os.path import basename

In [9]:
path_ref = '../../../SLIDE-x-D003-Arm-Thumb-Atmega-Leon3-RiscV-KERNEL/SLIDE-x-RES/KERNEL_DECIMAL'
path_ref_name = 'Thumb-Arm-Atmega-Leon3-RiscV-KERNEL-DECIMAL-'

In [10]:
import time
import pandas as pd
from IPython.core.magic import register_cell_magic
from IPython.display import display, HTML

# CSV file to save execution times
csv_filename = path_ref_name + "04-00-ExecutionTime.csv"

# Initialize the CSV file with header if it doesn't exist
try:
    df = pd.read_csv(csv_filename)
except FileNotFoundError:
    df = pd.DataFrame(columns=["cell_number", "execution_time"])
    df.to_csv(csv_filename, index=False)

# Function to log execution times
def log_execution_time(cell_number, execution_time):
    df = pd.read_csv(csv_filename)
    new_row = pd.DataFrame({"cell_number": [cell_number], "execution_time": [execution_time]})
    df = pd.concat([df, new_row], ignore_index=True)
    df.to_csv(csv_filename, index=False)

# Decorator for the magic function
@register_cell_magic
def log_time(line, cell):
    start_time = time.time()
    exec(cell, globals())
    end_time = time.time()
    execution_time = end_time - start_time

    # Current cell number
    cell_number = int(line) if line.isdigit() else None
    if cell_number is None:
        display(HTML("<b>Warning:</b> Cell number not provided!"))
        return

    # Log the execution time
    log_execution_time(cell_number, execution_time)
    display(HTML(f"<b>Cell {cell_number}:</b> Execution time = {execution_time:.6f} seconds"))

# To use the magic function in the notebook
# %%log_time 1
# cell code

In [11]:
%%log_time 1

# DataFrame name json file
nameFile = []
pathFile = []
rootFile = []

path = os.getcwd()
for root, dirs, files in os.walk(path_ref):
    for file in files:
        if file.endswith(".json"):
            nameFile.append(file)
            pathFile.append(os.path.join(root, file))
            rootFile.append(root)
            
nameFileS = pd.Series(nameFile)
pathFileS = pd.Series(pathFile)
rootFileS = pd.Series(rootFile)

pd.options.display.max_colwidth = 100
print('nameFileS:'+nameFileS)
print('pathFileS:'+pathFileS)
print('rootFileS:'+rootFileS)

dfNF = pd.concat([nameFileS, pathFileS, rootFileS], axis=1)
dfNF.columns = ['FILE_NAME', 'FILE_PATH', 'FILE_ROOT']

0    nameFileS:parameters.json
1    nameFileS:parameters.json
2    nameFileS:parameters.json
3    nameFileS:parameters.json
4    nameFileS:parameters.json
5    nameFileS:parameters.json
6    nameFileS:parameters.json
7    nameFileS:parameters.json
8    nameFileS:parameters.json
dtype: object
0    pathFileS:../../../SLIDE-x-D003-Arm-Thumb-Atmega-Leon3-RiscV-KERNEL/SLIDE-x-RES/KERNEL_DECIMAL/f...
1    pathFileS:../../../SLIDE-x-D003-Arm-Thumb-Atmega-Leon3-RiscV-KERNEL/SLIDE-x-RES/KERNEL_DECIMAL/b...
2    pathFileS:../../../SLIDE-x-D003-Arm-Thumb-Atmega-Leon3-RiscV-KERNEL/SLIDE-x-RES/KERNEL_DECIMAL/b...
3    pathFileS:../../../SLIDE-x-D003-Arm-Thumb-Atmega-Leon3-RiscV-KERNEL/SLIDE-x-RES/KERNEL_DECIMAL/b...
4    pathFileS:../../../SLIDE-x-D003-Arm-Thumb-Atmega-Leon3-RiscV-KERNEL/SLIDE-x-RES/KERNEL_DECIMAL/s...
5    pathFileS:../../../SLIDE-x-D003-Arm-Thumb-Atmega-Leon3-RiscV-KERNEL/SLIDE-x-RES/KERNEL_DECIMAL/k...
6    pathFileS:../../../SLIDE-x-D003-Arm-Thumb-Atmega-Leon3-RiscV-KERNEL/SLID

In [12]:
dfRDC = pd.read_csv("RangeValueClassification_tot.csv", sep=';')
del dfRDC['Unnamed: 0']

In [13]:
dfRDC

,DEVICE,FUNCTION,DATA_TYPE,NUM_S,RANGE_S,NUM_SI,RANGE_SI,NUM_ARR,RANGE_ARR,NUM_MAT,RANGE_MAT
0,Arm,floydwarshall,int8_t,0,0,1,3,0,0,1,1
1,Arm,floydwarshall,int16_t,0,0,1,3,0,0,1,1
2,Arm,floydwarshall,int32_t,0,0,1,3,0,0,1,1
3,Arm,floydwarshall,int64_t,0,0,1,3,0,0,1,1
4,Thumb,floydwarshall,int8_t,0,0,1,3,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...
175,RiscV,gcd,int64_t,2,3,0,0,0,0,0,0
176,Leon3,gcd,int8_t,2,3,0,0,0,0,0,0
177,Leon3,gcd,int16_t,2,3,0,0,0,0,0,0
178,Leon3,gcd,int32_t,2,3,0,0,0,0,0,0


In [14]:
%%log_time 2

nameFileH = [] # Affinity
pathFileH = [] # Affinity
rootFileH = [] # Affinity
nameFileM = [] # cc4csValues
pathFileM = [] # cc4csValues 
rootFileM = [] # cc4csValues
nameFileS = [] # Size SW
pathFileS = [] # Size SW
rootFileS = [] # Size SW
#nameFileC = []
#pathFileC = []
#rootFileC = []
nameFileA = [] # area HW
pathFileA = [] # area HW
rootFileA = [] # area HW

# path = os.getcwd()
for root, dirs, files in os.walk(path_ref):
    for file in files:
        if file.endswith("Affinity.csv"):
            nameFileH.append(file)
            pathFileH.append(os.path.join(root, file))
            rootFileH.append(root)
        elif file.endswith("cc4csValues.csv"):
            nameFileM.append(file)
            pathFileM.append(os.path.join(root, file))
            rootFileM.append(root)
        elif file.endswith("Size.csv"):
            nameFileS.append(file)
            pathFileS.append(os.path.join(root, file))
            rootFileS.append(root)
        elif file.endswith("idAreaValuesParam.csv"):
            nameFileA.append(file)
            pathFileA.append(os.path.join(root, file))
            rootFileA.append(root)

#print('nameFileH'+str(nameFileH))
#print('pathFileH'+str(pathFileH))
#print('rootFileH'+str(rootFileH))

#print('nameFileM'+str(nameFileM))
#print('pathFileM'+str(pathFileM))
#print('rootFileM'+str(rootFileM))

#print('nameFileS'+str(nameFileH))
#print('pathFileS'+str(nameFileH))
#print('rootFileS'+str(nameFileH))

#print('nameFileA'+str(nameFileH))
#print('pathFileA'+str(nameFileH))
#print('rootFileA'+str(nameFileH))

In [15]:
%%log_time 3

# Affinity file
for i in range(0, len(pathFileH)):
    path = pathFileH[i]
    try:
        dfHTemp = pd.read_csv(path, sep=',',engine='python')
    except:
        print(path)
        
    if os.name == 'nt':
        spR = rootFileH[i].split('\\')
    else:
        spR = rootFileH[i].split('/')
        
    matrixVectorSize = spR[-1] 
    optFlag = spR[-2]
    datatype = spR[-3]
    device = spR[-4]
    # board = 'xc7a100t-1csg324-Artix-7-200-MHz'  # Bambu (HW)
    board = spR[-4] # SW
    function = spR[-5]
    dfHTemp['DEVICE'] = device
    dfHTemp['BOARD'] = board
    dfHTemp['FUNCTION'] = function
    dfHTemp['DATA_TYPE'] = datatype
    dfHTemp['OPT_FLAG'] = optFlag
    dfHTemp['MV_SIZE'] = matrixVectorSize
    dfHTemp['inputName'] = dfHTemp['inputName'].map(lambda x: x.lstrip('values_'))
    dfHTemp['inputName'] = dfHTemp['inputName'].astype(int)
    dfHTemp.sort_values('inputName')
    
    if i == 0:
        dfHTot = dfHTemp
    else:
        frames = [dfHTot, dfHTemp]
        dfHTot = pd.concat(frames)
        
# a = dfHTot['Program_Volume']/3000
# a = a.values.tolist()
# dfHTot['Bugs_Delivered'] = a

In [16]:
dfHTot

,inputName,Total_Operators,Distinct_Operators,Total_Operands,Distinct_Operands,Program_Length,Vocabulary_Size,Program_Volume,Effort,Program_Level,...,BMR,DR_bit,Agpp,Aasp,DEVICE,BOARD,FUNCTION,DATA_TYPE,OPT_FLAG,MV_SIZE
0,11,1177,26,727,689,1904,715,18053.35,247637.41,0.07,...,0.000000,0.039890,0.499899,0.114840,Leon3,Leon3,floydwarshall,double,optO5-Ofast,32x32
1,19,1555,26,951,913,2506,939,24746.70,335096.92,0.07,...,0.000000,0.030494,0.480468,0.102745,Leon3,Leon3,floydwarshall,double,optO5-Ofast,32x32
2,12,214,26,87,49,301,75,1874.87,43275.16,0.04,...,0.000000,0.333333,0.834750,0.594739,Leon3,Leon3,floydwarshall,double,optO5-Ofast,32x32
3,38,553,26,307,269,860,295,7055.93,104684.84,0.07,...,0.000000,0.094463,0.600707,0.195989,Leon3,Leon3,floydwarshall,double,optO5-Ofast,32x32
4,81,1461,26,892,854,2353,880,23015.54,312515.45,0.07,...,0.000000,0.032511,0.484686,0.105290,Leon3,Leon3,floydwarshall,double,optO5-Ofast,32x32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,3,142,25,47,17,189,42,1019.15,35220.56,0.03,...,0.007042,0.595745,0.937733,0.787545,RiscV,RiscV,gcd,float,optO2-07,32x32
96,98,142,25,47,17,189,42,1019.15,35220.56,0.03,...,0.007042,0.595745,0.937733,0.787545,RiscV,RiscV,gcd,float,optO2-07,32x32
97,29,142,25,47,17,189,42,1019.15,35220.56,0.03,...,0.007042,0.595745,0.937733,0.787545,RiscV,RiscV,gcd,float,optO2-07,32x32
98,80,142,25,47,17,189,42,1019.15,35220.56,0.03,...,0.007042,0.595745,0.937733,0.787545,RiscV,RiscV,gcd,float,optO2-07,32x32


In [17]:
%%log_time 4

# Define the function to categorize clock cycles
def categorize_opt(clockCycles, minCC, q1CC, medCC, q3CC, maxCC):
    if clockCycles == minCC:
        return 1
    elif  clockCycles > minCC and clockCycles < q1CC:
        return 2
    elif clockCycles >= q1CC and clockCycles < medCC:
        return 3
    elif clockCycles >= medCC and clockCycles < q3CC:
        return 4
    elif clockCycles >= q3CC and clockCycles < maxCC:
        return 5
    elif clockCycles == maxCC:
        return 6
    
def has_number(column):
    # Use apply to check each value and any to see if any are numeric
    return column.apply(lambda x: isinstance(x, (int, float))).any()

# cc4csValues file
for i in range(0, len(pathFileM)):
    path = pathFileM[i]
    dfMTemp = pd.read_csv(path, sep=',', quotechar='"',engine='python')
        
    if os.name == 'nt':
        spR = rootFileM[i].split('\\')
    else:
        spR = rootFileM[i].split('/')  
    
    matrixVectorSize = spR[-1] 
    optFlag = spR[-2]
    datatype = spR[-3]
    device = spR[-4]
    # board = 'xc7a100t-1csg324-Artix-7-200-MHz'  # spR[-4] 
    board = spR[-4] 
    function = spR[-5]
    dfMTemp['DEVICE'] = device
    dfMTemp['BOARD'] = board
    dfMTemp['FUNCTION'] = function
    dfMTemp['DATA_TYPE'] = datatype
    dfMTemp['OPT_FLAG'] = optFlag
    dfMTemp['MV_SIZE'] = matrixVectorSize
    dfMTemp['inputName'] = dfMTemp['id'].map(lambda x: x.lstrip('values_'))
    dfMTemp['inputName'] = dfMTemp['inputName'].astype(int)
    dfMTemp.sort_values('inputName')
    dfMTemp.drop('id', axis=1, inplace=True)
    
    if (not has_number(dfMTemp['clockCycles'])):
        dfMTemp['clockCycles'] = 0
    
    if i == 0:
        dfMTot = dfMTemp
    else:
        frames = [dfMTot, dfMTemp]
        dfMTot = pd.concat(frames)

In [18]:
dfMTot

,clockCycles,assemblyInstr,cacheHit,cacheHitInstr,cacheHitData,MOPS,MIPS,MFLOPS,cInstr,cGimple,...,cc4ssa,cc4opt,DEVICE,BOARD,FUNCTION,DATA_TYPE,OPT_FLAG,MV_SIZE,inputName,Cache
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,73119,674215,...,0.000,0.000,Leon3,Leon3,floydwarshall,double,optO5-Ofast,32x32,11,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,111557,1031659,...,0.000,0.000,Leon3,Leon3,floydwarshall,double,optO5-Ofast,32x32,19,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1017,8243,...,0.000,0.000,Leon3,Leon3,floydwarshall,double,optO5-Ofast,32x32,12,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17329,155705,...,0.000,0.000,Leon3,Leon3,floydwarshall,double,optO5-Ofast,32x32,38,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101042,934605,...,0.000,0.000,Leon3,Leon3,floydwarshall,double,optO5-Ofast,32x32,81,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1191.0,1191.0,NaN,NaN,NaN,NaN,NaN,NaN,229,383,...,1.847,1.005,RiscV,RiscV,gcd,float,optO2-07,32x32,3,"['958316', '8845066', '126315', '1108968', '65031', '552252', '568389', '49.971', '12589748', '0..."
96,2395.0,2395.0,NaN,NaN,NaN,NaN,NaN,NaN,823,1268,...,0.783,0.419,RiscV,RiscV,gcd,float,optO2-07,32x32,98,"['958316', '8845066', '126315', '1108968', '65031', '552252', '568389', '49.971', '12593356', '0..."
97,1179.0,1179.0,NaN,NaN,NaN,NaN,NaN,NaN,235,401,...,1.926,1.054,RiscV,RiscV,gcd,float,optO2-07,32x32,29,"['958316', '8845066', '126315', '1108968', '65031', '552252', '568389', '49.971', '12589718', '0..."
98,1279.0,1279.0,NaN,NaN,NaN,NaN,NaN,NaN,277,458,...,1.564,0.848,RiscV,RiscV,gcd,float,optO2-07,32x32,80,"['958316', '8845066', '126315', '1108968', '65031', '552252', '568389', '49.971', '12590014', '0..."


In [19]:
%%log_time 5

# Size SW file
for i in range(0, len(pathFileS)):
    path = pathFileS[i]
    dfSTemp = pd.read_csv(path, sep=',',engine='python')

    if os.name == 'nt':
        spR = rootFileS[i].split('\\')
    else:
        spR = rootFileS[i].split('/')   
        
    matrixVectorSize = spR[-1] 
    optFlag = spR[-2]
    datatype = spR[-3]
    device = spR[-4]
    board = spR[-4]
    function = spR[-5]
    dfSTemp['DEVICE'] = device
    dfSTemp['BOARD'] = board
    dfSTemp['FUNCTION'] = function
    dfSTemp['DATA_TYPE'] = datatype
    dfSTemp['OPT_FLAG'] = optFlag
    dfSTemp['MV_SIZE'] = matrixVectorSize
    dfSTemp['inputName'] = dfSTemp['inputName'].map(lambda x: x.lstrip('values_') if isinstance(x, str) else x)
    dfSTemp['inputName'] = dfSTemp['inputName'].astype(int)
    dfSTemp.sort_values('inputName')
    #dfSTemp.iloc[:,0] = dfSTemp.iloc[:, 0].map(lambda x: x.lstrip('values_'))
    #dfSTemp.iloc[:,0] = dfSTemp.iloc[:, 0].astype(int)
    #dfSTemp.sort_values(by=dfSTemp.columns[0])
    
    if i == 0:
        dfSTot = dfSTemp
    else:
        frames = [dfSTot, dfSTemp]
        dfSTot = pd.concat(frames)

In [20]:
# dfSTemp.drop(dfSTemp.index, inplace=True)
dfSTot

,inputName,text,data,bss,DEVICE,BOARD,FUNCTION,DATA_TYPE,OPT_FLAG,MV_SIZE
0,11,11584.0,5552.0,392.0,Leon3,Leon3,floydwarshall,double,optO5-Ofast,32x32
1,19,11584.0,7344.0,392.0,Leon3,Leon3,floydwarshall,double,optO5-Ofast,32x32
2,12,12752.0,432.0,392.0,Leon3,Leon3,floydwarshall,double,optO5-Ofast,32x32
3,38,12080.0,2192.0,392.0,Leon3,Leon3,floydwarshall,double,optO5-Ofast,32x32
4,81,11584.0,6872.0,392.0,Leon3,Leon3,floydwarshall,double,optO5-Ofast,32x32
...,...,...,...,...,...,...,...,...,...,...
95,3,5844.0,2508.0,1472.0,RiscV,RiscV,gcd,float,optO2-07,32x32
96,98,5844.0,2508.0,1472.0,RiscV,RiscV,gcd,float,optO2-07,32x32
97,29,5844.0,2508.0,1472.0,RiscV,RiscV,gcd,float,optO2-07,32x32
98,80,5844.0,2508.0,1472.0,RiscV,RiscV,gcd,float,optO2-07,32x32


In [21]:
%%log_time 7

# area parameters file
for i in range(0, len(pathFileA)):
    path = pathFileA[i]
    dfATemp = pd.read_csv(path, sep=',',engine='python')
    
    if os.name == 'nt':
        spR = rootFileA[i].split('\\')
    else:
        spR = rootFileA[i].split('/')
    matrixVectorSize = spR[-1] 
    optFlag = spR[-2]
    datatype = spR[-3]
    device = spR[-4]
    board = 'xc7a100t-1csg324-Artix-7-200-MHz'  # spR[-4] 
    function = spR[-5]
    dfATemp['DEVICE'] = device
    dfATemp['BOARD'] = board
    dfATemp['FUNCTION'] = function
    dfATemp['DATA_TYPE'] = datatype
    dfATemp['OPT_FLAG'] = optFlag
    dfATemp['MV_SIZE'] = matrixVectorSize
    dfATemp['inputName'] = dfATemp['ID_VAL'].map(lambda x: x.lstrip('values_'))
    dfATemp['inputName'] = dfATemp['inputName'].astype(int)
    dfATemp.sort_values('ID_VAL')
    
    if i == 0:
        dfATot = dfATemp
    else:
        frames = [dfATot, dfATemp]
        dfATot = pd.concat(frames)

b = dfATot['Clock_Cycles']/dfATot['Max_Freq']
b = b.values.tolist()
dfATot['Time'] = b

c = (30/dfATot['Max_Freq'])*dfATot['Clock_Cycles']
c = c.values.tolist()
dfATot['Energy'] = c

NameError: name 'dfATot' is not defined

In [22]:
dfATot

NameError: name 'dfATot' is not defined

In [23]:
%%log_time 8

dfHTot = dfHTot.reset_index(drop=True)
dfMTot = dfMTot.reset_index(drop=True)
dfSTot = dfSTot.reset_index(drop=True)
#dfCTot = dfCTot.reset_index(drop=True)
# dfATot = dfATot.reset_index(drop=True)

In [24]:
dfHTot

,inputName,Total_Operators,Distinct_Operators,Total_Operands,Distinct_Operands,Program_Length,Vocabulary_Size,Program_Volume,Effort,Program_Level,...,BMR,DR_bit,Agpp,Aasp,DEVICE,BOARD,FUNCTION,DATA_TYPE,OPT_FLAG,MV_SIZE
0,11,1177,26,727,689,1904,715,18053.35,247637.41,0.07,...,0.000000,0.039890,0.499899,0.114840,Leon3,Leon3,floydwarshall,double,optO5-Ofast,32x32
1,19,1555,26,951,913,2506,939,24746.70,335096.92,0.07,...,0.000000,0.030494,0.480468,0.102745,Leon3,Leon3,floydwarshall,double,optO5-Ofast,32x32
2,12,214,26,87,49,301,75,1874.87,43275.16,0.04,...,0.000000,0.333333,0.834750,0.594739,Leon3,Leon3,floydwarshall,double,optO5-Ofast,32x32
3,38,553,26,307,269,860,295,7055.93,104684.84,0.07,...,0.000000,0.094463,0.600707,0.195989,Leon3,Leon3,floydwarshall,double,optO5-Ofast,32x32
4,81,1461,26,892,854,2353,880,23015.54,312515.45,0.07,...,0.000000,0.032511,0.484686,0.105290,Leon3,Leon3,floydwarshall,double,optO5-Ofast,32x32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183495,3,142,25,47,17,189,42,1019.15,35220.56,0.03,...,0.007042,0.595745,0.937733,0.787545,RiscV,RiscV,gcd,float,optO2-07,32x32
183496,98,142,25,47,17,189,42,1019.15,35220.56,0.03,...,0.007042,0.595745,0.937733,0.787545,RiscV,RiscV,gcd,float,optO2-07,32x32
183497,29,142,25,47,17,189,42,1019.15,35220.56,0.03,...,0.007042,0.595745,0.937733,0.787545,RiscV,RiscV,gcd,float,optO2-07,32x32
183498,80,142,25,47,17,189,42,1019.15,35220.56,0.03,...,0.007042,0.595745,0.937733,0.787545,RiscV,RiscV,gcd,float,optO2-07,32x32


In [25]:
dfMTot

,clockCycles,assemblyInstr,cacheHit,cacheHitInstr,cacheHitData,MOPS,MIPS,MFLOPS,cInstr,cGimple,...,cc4ssa,cc4opt,DEVICE,BOARD,FUNCTION,DATA_TYPE,OPT_FLAG,MV_SIZE,inputName,Cache
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,73119,674215,...,0.000,0.000,Leon3,Leon3,floydwarshall,double,optO5-Ofast,32x32,11,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,111557,1031659,...,0.000,0.000,Leon3,Leon3,floydwarshall,double,optO5-Ofast,32x32,19,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1017,8243,...,0.000,0.000,Leon3,Leon3,floydwarshall,double,optO5-Ofast,32x32,12,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17329,155705,...,0.000,0.000,Leon3,Leon3,floydwarshall,double,optO5-Ofast,32x32,38,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101042,934605,...,0.000,0.000,Leon3,Leon3,floydwarshall,double,optO5-Ofast,32x32,81,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183495,1191.0,1191.0,NaN,NaN,NaN,NaN,NaN,NaN,229,383,...,1.847,1.005,RiscV,RiscV,gcd,float,optO2-07,32x32,3,"['958316', '8845066', '126315', '1108968', '65031', '552252', '568389', '49.971', '12589748', '0..."
183496,2395.0,2395.0,NaN,NaN,NaN,NaN,NaN,NaN,823,1268,...,0.783,0.419,RiscV,RiscV,gcd,float,optO2-07,32x32,98,"['958316', '8845066', '126315', '1108968', '65031', '552252', '568389', '49.971', '12593356', '0..."
183497,1179.0,1179.0,NaN,NaN,NaN,NaN,NaN,NaN,235,401,...,1.926,1.054,RiscV,RiscV,gcd,float,optO2-07,32x32,29,"['958316', '8845066', '126315', '1108968', '65031', '552252', '568389', '49.971', '12589718', '0..."
183498,1279.0,1279.0,NaN,NaN,NaN,NaN,NaN,NaN,277,458,...,1.564,0.848,RiscV,RiscV,gcd,float,optO2-07,32x32,80,"['958316', '8845066', '126315', '1108968', '65031', '552252', '568389', '49.971', '12590014', '0..."


In [26]:
dfSTot

,inputName,text,data,bss,DEVICE,BOARD,FUNCTION,DATA_TYPE,OPT_FLAG,MV_SIZE
0,11,11584.0,5552.0,392.0,Leon3,Leon3,floydwarshall,double,optO5-Ofast,32x32
1,19,11584.0,7344.0,392.0,Leon3,Leon3,floydwarshall,double,optO5-Ofast,32x32
2,12,12752.0,432.0,392.0,Leon3,Leon3,floydwarshall,double,optO5-Ofast,32x32
3,38,12080.0,2192.0,392.0,Leon3,Leon3,floydwarshall,double,optO5-Ofast,32x32
4,81,11584.0,6872.0,392.0,Leon3,Leon3,floydwarshall,double,optO5-Ofast,32x32
...,...,...,...,...,...,...,...,...,...,...
183495,3,5844.0,2508.0,1472.0,RiscV,RiscV,gcd,float,optO2-07,32x32
183496,98,5844.0,2508.0,1472.0,RiscV,RiscV,gcd,float,optO2-07,32x32
183497,29,5844.0,2508.0,1472.0,RiscV,RiscV,gcd,float,optO2-07,32x32
183498,80,5844.0,2508.0,1472.0,RiscV,RiscV,gcd,float,optO2-07,32x32


In [27]:
dfATot

NameError: name 'dfATot' is not defined

In [28]:
%%log_time 9

# To get the common columns between the DataFrames, we can use set intersection
common_columns = set(dfHTot.columns).intersection(set(dfMTot.columns)).intersection(set(dfSTot.columns))

# Convert the intersection to a list
common_columns = list(common_columns)

# Merge the DataFrames using the common columns
dfTot = pd.merge(dfHTot, dfMTot, on=common_columns, how='outer')
dfTot = pd.merge(dfTot, dfSTot, on=common_columns, how='outer') # SW
# dfTot = pd.merge(dfTot, dfATot, on=common_columns, how='outer') # HW

# Display the resulting DataFrame
# print(dfTot)

In [29]:
dfTot

,inputName,Total_Operators,Distinct_Operators,Total_Operands,Distinct_Operands,Program_Length,Vocabulary_Size,Program_Volume,Effort,Program_Level,...,OnceEe,CallExe,cc4cs,cc4ir,cc4ssa,cc4opt,Cache,text,data,bss
0,0,731,66,460,208,1191,274,9644.76,703881.73,0.01,...,78.95,60.0,30.691,7.610,12.187,12.490,NaN,1684.0,512.0,168.0
1,0,731,66,460,208,1191,274,9644.76,703881.73,0.01,...,78.95,60.0,30.689,7.609,12.186,13.322,NaN,1668.0,512.0,168.0
2,0,731,66,460,208,1191,274,9644.76,703881.73,0.01,...,78.95,60.0,29.773,7.382,11.823,12.116,NaN,1664.0,512.0,168.0
3,0,731,66,460,208,1191,274,9644.76,703881.73,0.01,...,78.95,60.0,30.396,7.537,12.070,12.012,NaN,1788.0,512.0,168.0
4,0,731,66,460,208,1191,274,9644.76,703881.73,0.01,...,78.95,60.0,29.017,7.195,11.571,12.702,NaN,1844.0,512.0,168.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186768,99,159,27,59,24,218,51,1236.59,41039.29,0.03,...,100.00,100.0,16.489,4.657,3.933,1.568,NaN,284.0,36.0,0.0
186769,99,159,27,59,24,218,51,1236.59,41039.29,0.03,...,100.00,100.0,16.637,4.699,3.968,1.582,NaN,292.0,36.0,0.0
186770,99,159,27,59,24,218,51,1236.59,41039.29,0.03,...,100.00,100.0,17.163,4.847,4.094,5.115,NaN,80.0,36.0,0.0
186771,99,159,27,59,24,218,51,1236.59,41039.29,0.03,...,100.00,100.0,16.637,4.699,3.968,1.582,NaN,292.0,36.0,0.0


In [30]:
dfTot = dfTot[ ['inputName','DEVICE','BOARD','FUNCTION','DATA_TYPE','OPT_FLAG','MV_SIZE'] + [ col for col in dfTot.columns if (col != 'inputName' and col != 'DEVICE' and col != 'BOARD' and col != 'FUNCTION' and col != 'DATA_TYPE' and col != 'OPT_FLAG' and col != 'MV_SIZE') ] ]

In [31]:
dfTot

,inputName,DEVICE,BOARD,FUNCTION,DATA_TYPE,OPT_FLAG,MV_SIZE,Total_Operators,Distinct_Operators,Total_Operands,...,OnceEe,CallExe,cc4cs,cc4ir,cc4ssa,cc4opt,Cache,text,data,bss
0,0,Arm,Arm,astar,double,optO1-00,32x32,731,66,460,...,78.95,60.0,30.691,7.610,12.187,12.490,NaN,1684.0,512.0,168.0
1,0,Arm,Arm,astar,double,optO1-01,32x32,731,66,460,...,78.95,60.0,30.689,7.609,12.186,13.322,NaN,1668.0,512.0,168.0
2,0,Arm,Arm,astar,double,optO1-02,32x32,731,66,460,...,78.95,60.0,29.773,7.382,11.823,12.116,NaN,1664.0,512.0,168.0
3,0,Arm,Arm,astar,double,optO1-03,32x32,731,66,460,...,78.95,60.0,30.396,7.537,12.070,12.012,NaN,1788.0,512.0,168.0
4,0,Arm,Arm,astar,double,optO2-00,32x32,731,66,460,...,78.95,60.0,29.017,7.195,11.571,12.702,NaN,1844.0,512.0,168.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186768,99,Thumb,Thumb,selectionsort,float,optO3-04,32x32,159,27,59,...,100.00,100.0,16.489,4.657,3.933,1.568,NaN,284.0,36.0,0.0
186769,99,Thumb,Thumb,selectionsort,float,optO3-05,32x32,159,27,59,...,100.00,100.0,16.637,4.699,3.968,1.582,NaN,292.0,36.0,0.0
186770,99,Thumb,Thumb,selectionsort,float,optO4-Os,32x32,159,27,59,...,100.00,100.0,17.163,4.847,4.094,5.115,NaN,80.0,36.0,0.0
186771,99,Thumb,Thumb,selectionsort,float,optO5-Ofast,32x32,159,27,59,...,100.00,100.0,16.637,4.699,3.968,1.582,NaN,292.0,36.0,0.0


In [32]:
dfTot = pd.merge(dfTot,dfRDC,how='left',on=['DEVICE','FUNCTION','DATA_TYPE'])

In [33]:
dfTot

,inputName,DEVICE,BOARD,FUNCTION,DATA_TYPE,OPT_FLAG,MV_SIZE,Total_Operators,Distinct_Operators,Total_Operands,...,data,bss,NUM_S,RANGE_S,NUM_SI,RANGE_SI,NUM_ARR,RANGE_ARR,NUM_MAT,RANGE_MAT
0,0,Arm,Arm,astar,double,optO1-00,32x32,731,66,460,...,512.0,168.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,Arm,Arm,astar,double,optO1-01,32x32,731,66,460,...,512.0,168.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,Arm,Arm,astar,double,optO1-02,32x32,731,66,460,...,512.0,168.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,Arm,Arm,astar,double,optO1-03,32x32,731,66,460,...,512.0,168.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,Arm,Arm,astar,double,optO2-00,32x32,731,66,460,...,512.0,168.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186768,99,Thumb,Thumb,selectionsort,float,optO3-04,32x32,159,27,59,...,36.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
186769,99,Thumb,Thumb,selectionsort,float,optO3-05,32x32,159,27,59,...,36.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
186770,99,Thumb,Thumb,selectionsort,float,optO4-Os,32x32,159,27,59,...,36.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
186771,99,Thumb,Thumb,selectionsort,float,optO5-Ofast,32x32,159,27,59,...,36.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
dfTot.rename(columns={'NUM_S':'SCALAR_INPUT',
                      'RANGE_S':'RANGE_SCALAR_VALUES',
                      'NUM_SI':'SCALAR_INDEX_INPUT',
                      'RANGE_SI':'RANGE_SCALAR_INDEX_VALUES',
                      'NUM_ARR':'ARRAY_INPUT',
                      'RANGE_ARR':'RANGE_ARRAY_INPUT',
                      'NUM_MAT':'MAT_INPUT',
                      'RANGE_MAT':'RANGE_MAT_INPUT'}, inplace=True)

In [36]:
%%log_time 10

dfTot.to_csv("TotalParameterMatrix.csv", sep=';', index=False, encoding='utf-8')

In [38]:
%%log_time 11

import pandas as pd
import os

def split_csv_by_column(input_file, column_name):
    # Read the CSV file
    df = pd.read_csv(input_file, sep=';')

    # Get unique values in the specified column
    unique_values = df[column_name].unique()

    # Get the directory of the input file
    input_dir = os.path.dirname(input_file)

    # For each unique value, create a subset of the DataFrame and save it as a CSV file
    for value in unique_values:
        subset_df = df[df[column_name] == value]
        output_file = os.path.join(input_dir, f"{value}-KERNEL-DECIMAL.csv")
        subset_df.to_csv(output_file, index=False)
        print(f"Created file: {output_file}")

input_file = 'Thumb-Arm-RiscV-Leon3-Atmega-KERNEL-DECIMAL.csv'  # replace with the path to your CSV file
column_name = 'DEVICE'  # replace with the name of the column to split by

split_csv_by_column(input_file, column_name)

<string>:7: DtypeWarning: Columns (131) have mixed types. Specify dtype option on import or set low_memory=False.


Created file: Arm-KERNEL-DECIMAL.csv
Created file: Atmega328p-KERNEL-DECIMAL.csv
Created file: Leon3-KERNEL-DECIMAL.csv
Created file: RiscV-KERNEL-DECIMAL.csv
Created file: Thumb-KERNEL-DECIMAL.csv


In [21]:
import pandas as pd
import os

def merge_csv_files(file1, file2):
    # Read the CSV files
    df1 = pd.read_csv(file1)
    df2 = pd.read_csv(file2)

    # Concatenate the DataFrames, axis=1 to concatenate columns
    merged_df = pd.concat([df1, df2], axis=1)

    # Get the directory of the input files
    input_dir = os.path.dirname(file1)

    # Define the output file path
    output_file = os.path.join(input_dir, 'xc7a100t-1csg324-Artix-7-200-MHz-RECIPE-DECIMAL.csv')

    # Save the merged DataFrame to a new CSV file
    merged_df.to_csv(output_file, index=False)
    print(f"Merged file saved as: {output_file}")

# Example usage
file1 = 'xc7a100t-1csg324-Artix-7-200-MHz-RECIPE-DECIMAL-1.csv'  # replace with the path to your first CSV file
file2 = 'xc7a100t-1csg324-Artix-7-200-MHz-RECIPE-DECIMAL-2.csv'  # replace with the path to your second CSV file

merge_csv_files(file1, file2)

Merged file saved as: xc7a100t-1csg324-Artix-7-200-MHz-RECIPE-DECIMAL.csv
